In [1]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

from KETIPrePartialDataPreprocessing import makeNaNImputationTest

inputType ='file' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}
imputation_param ={
"imputation_method":[
    {"min":0,"max":2,"method":"linear"}],"totalNanLimit":90}

# mean, median X
# ffill~polynomial_interpolate=> 각 구간마다 딱 고만큼만 Fill

input_data = makeNaNImputationTest.inputControl(inputType)
from KETIPrePartialDataPreprocessing import data_preprocessing
import numpy as np
input_test_data = input_data.replace(4.0, np.nan)
output = data_preprocessing.get_preprocessed_data(input_test_data[:25], refine_param, outlier_param, imputation_param)

Preprocessing:Refining
Preprocessing:DataWithMoreNaN
Preprocessing:Imputation
NaN Ratio temp 60.86956521739131
NaN Ratio co2 30.434782608695656
NaN Ratio pm10 34.78260869565217


In [2]:
for key in output:
    print(key, output[key][:20])

original                      temp  co2    pm10
timedate                              
2021-01-28 12:51:00   3.0  5.0 -9999.0
2021-01-28 12:52:00   3.0  5.0    64.0
2021-01-28 12:53:00   NaN  6.0    56.0
2021-01-28 12:54:00   NaN  7.0    59.0
2021-01-28 12:55:00   NaN  7.0    59.0
2021-01-28 12:56:00   3.0  6.0    58.0
2021-01-28 12:57:00   NaN  7.0    57.0
2021-01-28 12:58:00   NaN  7.0    58.0
2021-01-28 12:59:00   NaN  6.0    57.0
2021-01-28 13:00:00   NaN  7.0    56.0
2021-01-28 13:01:00   NaN  6.0    58.0
2021-01-28 13:02:00   NaN  7.0    57.0
2021-01-28 13:03:00   NaN  6.0    56.0
2021-01-28 13:04:00   NaN  6.0    57.0
2021-01-28 13:05:00   3.0  6.0    59.0
2021-01-28 13:06:00   3.0  6.0    61.0
2021-01-28 13:07:00   3.0  6.0    59.0
2021-01-28 13:08:00   3.0  6.0    61.0
2021-01-28 13:09:00   3.0  5.0    60.0
2021-01-28 13:10:00   3.0  6.0    60.0
refined_data                      temp  co2    pm10
timedate                              
2021-01-28 12:51:00   3.0  5.0 -9999.0
202